In [ ]:
'''
                                      ----------------------------------------

                                           COMPREHENSIVE EXAM ARTIFACT


                                           Author: Yao Gahounzo
                                                   Computing PhD 
                                                   Computational Math, Science and Engineering
                                                   Boise State University

                                           Date: September 29, 2021

                                      ----------------------------------------

    This program call the modules Module_ice_ocean and Inputs. The Inputs module contains exact solution, domain
    and some other functions. The different solvers are call from the Module_ice_ocean.
    
    The program contains two test cases: 'unit' or 'ice-ocean'.
    To switch from unit test to ice-ocean, you have to change: text_case = 'unit' or 'ice-ocean'
    
    In the case of unit test, one need to choose the problem boundary conditions (BCs), namely Dirichlet, 
    Neumann, and Robin. So problem with

    Dirichlet BC is: icase = 1
    Neumann BC is: icase = 2
    Robin BC is: icase = 3

    
    Order           : contains the list of the polynomial order used in the numerical integration
    N_element       : contains the number of elements in the domain
    time_method     : time integration method
    ti_method       : stages of the time integration method (implicit Runge Kutta method)
    icase           : select the time of the boundary conditions you want to test
    method_type     : continous galerkin (cg) method, it is the only one implemented in Module_ice_ocean
    integration_type: exact or inexact integration
    text_case       : select either we are in unit test case or the ice-ocean simulation

'''

%matplotlib notebook
%pylab
import pandas as pd
from time import perf_counter
from scipy.optimize import fsolve
from scipy.special import erf


# import module that contains the functions and solver
from Module_ice_ocean import*

# import Inputs module
from Inputs import*




# Initialization

ti_method = 3                 # Stages of IRK methods
time_method = "BDF3"          # IRK = implicit RK method, BDF2 or BDF3
integration_type = 1          # % = 1 is inexact and = 2 is exact
method_type = 'cg'            # Only CG method in the module (Module_ice_ocean)
iplot = False                 # plot the solution
icase = 1                     # equation with: 1 = Dirichlet BC, 2 = Neumann BC, 3 = Robin BC

text_case = 'ice-ocean'       # unit: ideal test case for convergence studies
                              # ice-ocean: for ice ocean simulation
    
if(text_case == 'unit'):      # ideal test case for convergence studies

    # diffusion coefficients (diffusivity)
    c_diff = coeff_diffu(icase)

    # Boundary conditition type                   

    # Robin: alpha = 1, beta != 0
    # Neumann: alpha = 1, beta = 0
    # Dirichlet: alpha = 0, beta = 1

    alpha, beta = boundary_conditions_coeff(icase)    # type of BC

    # Problem domain
    ax, bx = domain(icase)
    
    # Initialization
    TW = array([1])
    
    order = array([1,2,3,4])            # polynomial order
    N_element = array([8,16,32,64])     # Number of elements
    
    Tfinal = 0.5                        # Duration of the simulation in unit test
    
    len_el = len(N_element)
    len_pol = len(order)
    l2e_norm = zeros((len_pol, len_el))
    max_norm = zeros((len_pol, len_el))

    Np_array = zeros((len_pol, len_el))  
    
elif(text_case == 'ice-ocean'):    # ice-ocean: for ice ocean simulation
    
    iplot = True                   # plot the solution
    
    TW = array([0.3,2.3,5.4])
    
    # Simulation domain
    ax = 0
    bx = 0.5
    
    Tfinal = 42                   # Duration of the simulation
    
    order = array([2])            # polynomial order
    N_element = array([256])      # Number of element in the domain

# Initialization
u = 1.5
Nv = N_element

In [ ]:
# Load the data from the thermal picture in Gayen et al. (2016) 

# Data for far-field temperature (Tw) equal to 0.3
data03 = loadtxt('tempData03_1.csv',delimiter=',')
x03 = data03[:,0]
y03 = data03[:,1]

# Data for far-field temperature (Tw) equal to 2.3
data203 = loadtxt('tempData203.csv',delimiter=',')
x203 = data203[:,0]
y203 = data203[:,1]

# Data for far-field temperature (Tw) equal to 5.4
data504 = loadtxt('tempData504_1.csv',delimiter=',')
x504 = data504[:,0]
y504 = data504[:,1]

# Array of that contains the data for ploting purpose
X = [x03,x203,x504]
Y = [y03,y203,y504]

In [ ]:
'''
Simulations

'''


# Begining of the simulation

for k,Tw in enumerate(TW):                # Loop through the value ambient temperature

    for iN,N in enumerate(order):

        CFL = 1/(N+1)                     # CFL number

        if (integration_type == 1):       # Inexact integration
            Q = N
        elif (integration_type == 2):     # Exact integration
            Q = N+1

        wall = 0

        for e, nel in enumerate(Nv):

            Np = nel*N + 1                # Global number of grid points in the domain
            
            # Call of 1D diffusion solver
            '''
            outputs:
            --------
            qe         : Exact solution
            q          : Numerical solution
            coord      : All grid points
            intma      : Intma(CG/DG)
            '''
            tic = perf_counter()

            if(text_case == 'unit'):            # Unit test case
                
                # Call of the diffusion solver
                qe, q,coord, intma, tf = diff_Solver(N,Q,nel, Np, ax, bx, integration_type, g, exactSol,\
                                           c_diff,u,CFL, Tfinal, method_type, icase, alpha, beta,\
                                                 ti_method, time_method)
                
                print("\twalltime = {:e}".format(tf))


                # Compute L2- norm
                num = sum((q-qe)**2)
                denom = sum(qe**2 )

                e2 = sqrt(num/denom)
                l2e_norm[iN,e] = e2
                # Compute max-norm
                max_norm[iN, e] = max(abs(q-qe))
                # Store global number of grid point
                Np_array[iN,e] = Np           
                
            elif(text_case == 'ice-ocean'):    # Ice-ocean 

                
                # Call the ice-ocean solver from Inputs module
                
                '''
                outputs:
                --------
                S          : Salinity
                T          : Temperature
                coord      : All grid points
                intma      : Intma(CG/DG)
                '''
                
                S,T,coord,intma,tf = ice_simulation(N,Q,nel,Np, ax, bx, integration_type,method_type,\
                                                    ti_method, time_method,CFL,Tw,Tfinal,u)

                print("\twalltime = {:e}".format(tf))

                toc = perf_counter()
                wall += toc - tic
                
            # Form the global grid points for ploting purpose
            
            x_sol = zeros(Np)
            for ie in range(1,nel+1):
                for i in range(N+1):
                    ip = int(intma[i,ie-1])
                    x_sol[ip] = coord[i,ie-1]
                    
            
        # Plots
        if(iplot == True and text_case == 'ice-ocean'):
            
            # interpolate the data loaded from Gayen et al. 2016
            Tgayen = interp(x_sol, X[k], Y[k])
            
            indx = abs(x_sol-0.051).argmin()   # zoom the results near the interface [0,0.05]
            indx1 = abs(x_sol-0.0162).argmin()   # zoom the results near the interface [0,0.05]
               
            # Plot the temperature behavior
            figure(1)
            rcParams.update({'font.size': 12})
            p1, = plot(x_sol[:indx], T[:indx], '-', label = 'Tw = {}'.format(Tw))
            p2, = plot(x_sol[:indx], Tgayen[:indx], ':')
            ylim([-0.5,5.5])
            xlabel('x(m)')
            ylabel('Temperature (˚C)')
            title('Temperature profile')
            grid(linestyle = '--', linewidth = 0.5)
            leg1 = legend(title = 'Far-field temperature',loc = 1)
            gca().add_artist(leg1)
            leg2 = legend([p2],['Gayen et al. 2016'],loc = 4)
            gca().add_artist(leg1)
            
            # Plot the salinity behavior
            figure(2)
            plot(x_sol[:indx], S[:indx], ':', label = '{}'.format(Tw))
            xlabel('x(m)')
            ylabel('Salinity (psu)')
            title('Salinity profile')
            grid(linestyle = '--', linewidth = 0.5)
            legend(title = 'Far-field temperature')

# End of the simulation

In [ ]:
# Plot convergence if unit test case
if(text_case == 'unit'):
    
    import cg_graphics           # import cg_graphics module
    rcParams.update({'font.size': 12})
    
    figure(1)
    plot(x_sol,q, '-', label = 'Computed')
    plot(x_sol,qe, '--', label = 'Exact') 
    xlabel('x')
    ylabel('Solutions')
    title('Exact and Computed ({}) solutions: Time = {}'.format('cg'.upper(), Tfinal))
    grid(axis='both',linestyle='--')
    legend()
    show()   
    
    figure(2)
    clf()

    for i,N in enumerate(order):

        if(N >= 3):
            p = polyfit(log(Nv[:2]), log(l2e_norm[i][:2]), 1)
        else:

            p = polyfit(log(Nv), log(l2e_norm[i]), 1)

        loglog(Nv, l2e_norm[i], '-o',markersize=5, label = 'N = {:d}: rate = {:.2f}'.format(N,p[0]))

        loglog(Nv, exp(polyval(p,log(Nv))), '--')

    cg_graphics.set_xticks(Nv)
    xlabel('# Elements')
    ylabel('Error (L2-error)')
    title('Error vs number of Elements ({:s}, {:s})'.format('cg'.upper(), time_method))
    grid(axis='both',linestyle='--')
    legend()
    show()   

In [ ]:
if(text_case == 'ice-ocean'):
    
    figure(3)
    
    SW = array([10,15,20,25,35])

    TW = array([0.3,2.3,5.4])

    V = zeros(len(SW))
    
    
    for j,Tw in enumerate(TW):
        
        for i,Sw in enumerate(SW):

            L = coefF(Tw, gammaS,gammaT, cw, Li, cI, TS, b, c, pb, a, Sw)

            # compute salinity at the boundary
            SB1, SB2 = SaltB(K,L,M,Sw)
            
            

            # Compute melt rate
            V[i] = Meltrate(Sw, SB2, gammaS)
    
        V = 1e6*V

        plot(SW,V,'-o',label = 'Tw = {}'.format(Tw))
        legend()
        xlabel('Salinity (psu)')
        ylabel('Melting rate V ($\mu ms^{-1}$)')
        grid(linestyle = '--', linewidth = 0.5)

        
    figure(4)
    
    Sw = 35
    P = 1000
    
    # Formulas to compute freezing temperature
    # http://www.code10.info/index.php?option=com_content&view=article&id=66:
    # calculating-the-freezing-point-of-seawater&catid=54:cat_coding_algorithms_seawater&Itemid=79
    TL = (-0.0575 + 1.710523e-3*sqrt(abs(Sw)) - 2.154996e-4*Sw)*Sw - 7.53e-4*P

    TW = array([0,0.3,2.3,3,4.5,5.2,5.8]) - TL
    TB = zeros(len(TW))
    
    for i,Tw in enumerate(TW):
        
        L = coefF(Tw, gammaS,gammaT, cw, Li, cI, TS, b, c, pb, a, Sw)
        
        SB1, SB2 = SaltB(K,L,M,Sw)
        
        TB[i] = a*SB2 + b + c*pb
        
    plot(TW,TB,'-o')
    xlabel('$T_w-T_L$')
    ylabel('Interface temperature $T_i$(˚C)')
    grid(linestyle = '--', linewidth = 0.5)   